# paiza 練習問題で学ぶアルゴリズム
## [日別訪問者数の最大平均区間(large) (paizaランク A 相当)](https://paiza.jp/works/mondai/skillcheck_archive/max_range_large)を解いてみよう！

Bランクの練習問題はクリア出来ましたが、同じ内容のAランク問題では100点を取ることが出来ませんでした。  
違いを調べ、ソースコードを改善してAランク問題も攻略しましょう！

### 1. 計算量を調べてみよう

計算量とは、プログラムを実行した時の、大雑把な計算回数のことです。  
入力例1、入力例2で計算の動きを見てみましょう。

<br>
<div style="text-align: center;">
  入力例1 計算する区間(ログ: n = 5, キャンペーン: k = 3)
</div>

|入力例1|day1|day2|day3|day4|day5|
|:--:|:--:|:--:|:--:|:--:|:--:|
|訪問者数|1|2|3|2|1|
|キャンペーン候補1|1|2|3|||
|キャンペーン候補2||2|3|2||
|キャンペーン候補3|||3|2|1|

<br>
<div style="text-align: center;">
    キャンペーン候補区間数: 3、区間の要素数: 3、計算回数 3 × 3 = 9回
</div>
<br>

---

<div style="text-align: center;">
  入力例2 計算する区間(ログ: n = 10, キャンペーン: k = 2)
</div>

|入力例2|day1|day2|day3|day4|day5|day6|day7|day8|day9|day10|
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|訪問者数|6|2|0|7|1|3|5|3|2|6|
|キャンペーン候補1|6|2|||||||||
|キャンペーン候補2||2|0||||||||
|キャンペーン候補3|||0|7|||||||
|キャンペーン候補4||||7|1||||||
|キャンペーン候補5|||||1|3|||||
|キャンペーン候補5||||||3|5||||
|キャンペーン候補5|||||||5|3|||
|キャンペーン候補5||||||||3|2||
|キャンペーン候補6|||||||||2|6|

<br>
<div style="text-align: center;">
    キャンペーン候補区間数: 6、区間の要素数: 2、計算回数 6 × 2 = 12回
</div>
<br>

#### 計算回数についてまとめる

- 計算回数 = (n - k + 1) * k

- n が一定の場合、k が n / 2 のとき、計算量が一番大きくなる
  - n=5, k=1  ->  (5 - 1 + 1) * 1 = 5
  - n=5, k=2  ->  (5 - 2 + 1) * 2 = 8
  - **n=5, k=3  ->  (5 - 3 + 1) * 3 = 9**
  - n=5, k=4  ->  (5 - 4 + 1) * 4 = 8
  - n=5, k=5  ->  (5 - 5 + 1) * 5 = 5

タイムオーバーのケース n = 300000、k = 150000 だと…

**n=300000, k=150000  ->  (300000 - 150000 + 1) * 150000 = 22,500,150,000（225億15万回!!）**  

※ ちなみに判定では１６秒で処理が打ち切られましたが、プログラムを最後まで実行すると３６秒かかりました。

---

### 2. Bランク問題とAランク問題の違いについて

Bランク問題では、ログデータの件数が最大1000件でしたが、Aランク問題では最大30万件となっています。  
データ件数が少ないうちは処理の流れが正しく書けていれば問題はないのですが、処理件数が多くなると、制限時間内で処理しきれないケースが出てきます。（paizaではタイムオーバー、AtCoderではTime Limit Exceededという失敗になります）  
これは、出題側が想定した計算量をオーバーしているということになりますので、そのへんを解決する工夫がAランク以上では求められます。

さて、計算数を減らす工夫をしなければいけないのですが、自力ではなかな思いつけないですよねぇ...（本番問題だと時間制限もあるし）  
そこで使われるのが、「代表的なアルゴリズム」の数々というわけです！  
「ユークリッドの互除法」とか、「エラトステネスの篩（ふるい）」など聞いたことないでしょうか？

Aランク以上では、そのような代表的な「アルゴリズム」を、問題に合わせて選択し、応用する能力が必要というわけです。  
数学で言えば公式を覚えて応用するようなイメージでしょうかね。

#### 代表的なアルゴリズム
- 貪欲法
- 分割統治法
- 動的計画法
- 二分探索法
- 幅優先探索
- 深さ優先探索

---

### 3. プログラムの改善を考える

#### タイムオーバーしたプログラム

```ruby
# データ入力
log_count, campaign_days = gets.split.map(&:to_i)
visitor_count = gets.split.map(&:to_i)

# 区間平均を求める
section_averages = []

# 【 n-k+1 回のループ 】
(0..log_count - campaign_days).each do |idx|
    
  # 【 k 回のループ 】
  section_averages << visitor_count[idx..idx + campaign_days - 1].sum.fdiv(campaign_days)
end

# 【 ここから先は計算量の対象としていません 】

# maxメソッドで、最大平均(max_average)を求める
max_average = section_averages.max
# countメソッドで、最大平均の数(キャンペーン候補)を求める
candidate_count = section_averages.count(max_average)
# find_indexメソッドで、キャンペーン最初の候補日を求める(index 0 が 1日目なので index + 1 とする)
first_day = section_averages.find_index {|average| average == max_average} + 1
# 結果を出力
puts "#{candidate_count} #{first_day}"
```

#### 改善したプログラム

```ruby
# データ入力
log_count, campaign_days = gets.split.map(&:to_i)
visitor_counts = gets.split.map(&:to_i)

# 初回のキャンペーン候補区間の合計を求める　【 k 回のループ 】
section_visitors = [visitor_counts[0..campaign_days - 1].sum]

# 次のキャンペーン候補区間からの処理 【 n - k 回のループ】
(1..log_count - campaign_days).each do |current_idx|
    
  # キャンペーン候補から除く訪問者数　【 1回 】
  remove_idx = current_idx - 1
  # キャンペーン候補日に加える訪問者数　【 1回 】
  insert_idx = current_idx + campaign_days - 1

  # 今回のキャンペーン区間の訪問者数を計算する　【 1回 】
  section_visitors << section_visitors[-1] - visitor_counts[remove_idx] + visitor_counts[insert_idx]
end

# 【 ここから先は計算量の対象としていません 】

# maxメソッドで、最大来場者数を求める
max_visitors = section_visitors.max

# countメソッドで、最大来場者数だった日数(キャンペーン候補)を求める
candidate_count = section_visitors.count(max_visitors)
# find_indexメソッドで、キャンペーン最初の候補日を求める(index 0 が 1日目なので index + 1 とする)
first_day = section_visitors.find_index { |visitor_count| visitor_count == max_visitors } + 1

# 結果を返す
puts "#{candidate_count} #{first_day}"
```
---

ループ内でsumメソッドを使っていた部分を改善することで、計算量をかなり減らすことができました！

- 改善前の計算回数 = (n - k + 1) * k --> (300,000 - 150,000 + 1) * 150000 = 22,500,150,000
- 改善後の計算回数 = k + (n - k) * 3 --> 150,000 + (300,000 - 150,000) * 3 = 600,000

__※ 計算回数が約37500分の1になった！__



### 5. 結果

改善前は区間の和をsumメソッドで毎回計算していましたが、左端と右端の値に注目し、必要な計算だけを行うことによって計算量を37500分の1に減らすことが出来ました。（オーダーで言うと O(n^2) から O(n) になりました。）  
改善に使ったのは「しゃくとり法」というアルゴリズムです。左右の端が伸びたり縮んだりして、「しゃくとり虫」みたいですね。


### 6. まとめ

- paiza Aランク以上の問題は、実装力に加えて計算量削減の工夫が必要
- ループ（ネスト）が深くなると爆発的に計算量が増える
- 代表的なアルゴリズムを知っておくと、数学の公式のように使うことが出来る


#### 参考文献:

- [paiza開発日誌 【累積和、しゃくとり法】初級者でも解るアルゴリズム図解](https://paiza.hatenablog.com/entry/2015/01/21/%E3%80%90%E7%B4%AF%E7%A9%8D%E5%92%8C%E3%80%81%E3%81%97%E3%82%83%E3%81%8F%E3%81%A8%E3%82%8A%E6%B3%95%E3%80%91%E5%88%9D%E7%B4%9A%E8%80%85%E3%81%A7%E3%82%82%E8%A7%A3%E3%82%8B%E3%82%A2%E3%83%AB%E3%82%B4)